<a href="https://colab.research.google.com/github/piyushgoel997/MutationClassification/blob/master/Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports


In [0]:
import numpy as np
import pandas as pd
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Conv1D, BatchNormalization, GlobalMaxPooling1D
from sklearn.utils import shuffle

data_url = "https://raw.githubusercontent.com/piyushgoel997/MutationClassification/master/data.csv"

Load and process data to make matrix

In [7]:
def get_aa_encoder():
    """
    One-hot encoding indices for the 20 possible amino acids.
    :return: the dictionary mapping amino acid to index in the one-hot encoding.
    """
    acids = "ARNDCEQGHILKMFPSTWYV"
    indices = {}
    i = 0
    for a in acids:
        indices[a] = i
        i += 1
    return indices


def make_matrix(seq, alt):
    """
    makes a matrix where each of the 51 features correspond to the amino acid at
    that position in the sequence after the one-hot encoding using the indices
    provided. Also puts a -1 at in the middle most feature at the position
    corresponding to the alt amino acid.
    :param seq: the sequence to be encoded.
    :param alt: the alternate amino acid
    :return: the resultant encoded matrix (51 X 20)
    """
    indices = get_aa_encoder()
    matrix = np.zeros((len(indices), len(seq)))
    i = 0
    for s in seq:
        matrix[indices[s]][i] = 1
        i += 1
    matrix[indices[alt]][int(len(seq) / 2)] = -1
    return matrix.T


def map_description(description):
    map = {'Benign': 0, 'Benign/Likely benign': 0, 'Likely benign': 0,
           'Likely pathogenic': 1, 'Pathogenic': 1, 'Pathogenic/Likely pathogenic': 1}
    return map[description]


X = []
Y = []
mutations = pd.read_csv(data_url)

for _, row in mutations.iterrows():
    X.append(make_matrix(row["REF_SEQ"], row["ALT_AA"]))
    Y.append(map_description(row["DESCRIPTION"]))

X = np.array(X)
Y = np.array(Y)

# reshaping to be able to apply convulutional layer with 1 channel.
# X = X.reshape(X.shape[0],X.shape[1], 1, X.shape[2])

print("Data shape", X.shape)
print("Output shape", Y.shape)

Data shape (229502, 51, 20)
Output shape (229502,)


In [0]:
model = Sequential()

model.add(Conv1D(32, kernel_size=3, activation="relu"))
model.add(Conv1D(64, kernel_size=5, activation="relu"))
model.add(BatchNormalization())
model.add(Conv1D(64, kernel_size=5, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(32, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(1, kernel_size=1, activation='sigmoid'))
model.add(BatchNormalization())
model.add(GlobalMaxPooling1D())

In [0]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [0]:
X, Y = shuffle(X, Y)
X_train = X[:int(0.8*X.shape[0])]
Y_train = Y[:int(0.8*Y.shape[0])]
X_test = X[int(0.8*X.shape[0]):]
Y_test = Y[int(0.8*Y.shape[0]):]

In [23]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20)

Epoch 1/20
5738/5738 [==============================] - 39s 7ms/step - loss: 0.1315 - accuracy: 0.8281 - val_loss: 0.0923 - val_accuracy: 0.8741
Epoch 2/20
5738/5738 [==============================] - 39s 7ms/step - loss: 0.0868 - accuracy: 0.8775 - val_loss: 0.0830 - val_accuracy: 0.8831
Epoch 3/20
5738/5738 [==============================] - 39s 7ms/step - loss: 0.0793 - accuracy: 0.8879 - val_loss: 0.0804 - val_accuracy: 0.8863
Epoch 4/20
5738/5738 [==============================] - 39s 7ms/step - loss: 0.0752 - accuracy: 0.8937 - val_loss: 0.0772 - val_accuracy: 0.8920
Epoch 5/20
5738/5738 [==============================] - 39s 7ms/step - loss: 0.0723 - accuracy: 0.8980 - val_loss: 0.0753 - val_accuracy: 0.8942
Epoch 6/20
5738/5738 [==============================] - 39s 7ms/step - loss: 0.0696 - accuracy: 0.9038 - val_loss: 0.0742 - val_accuracy: 0.8967
Epoch 7/20
5738/5738 [==============================] - 39s 7ms/step - loss: 0.0671 - accuracy: 0.9077 - val_loss: 0.0732 - val_ac